# Numerical Techniques 4: Spectral techniques



## Preparations: load numpy and matplotlib libraries

In [1]:
## Load numpy and matplotlib libraries
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.ioff()
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 100  # reduce to generate figures faster (but smaller)

# function for animating results
def create_animation(phi,frames=None,labels=None,ylim=None):
    
    # create figure with axes
    fig,ax=plt.subplots()
    
    if len(phi.shape)==2:
        phi=phi.reshape([phi.shape[0],phi.shape[1],1])
    
    # number of experiments
    nExp=phi.shape[2]

    # determine frames to plot
    if frames is None:
        # number of timesteps
        nt=phi.shape[0]-1
        frames=np.arange(0,nt+1)
    else:
        nt=frames[-1]
    
    
    # set y limits
    if ylim is None:
        ymin=np.min(phi[0,:,:])
        ymax=np.max(phi[0,:,:])
        ylim=[1.2*ymin-0.2*ymax,1.2*ymax-0.2*ymin]
    
    # labels and legend   
    showlegend=True
    if labels is None:
        showlegend=False
        labels=['exp %i'%jExp for jExp in range(nExp)]
        
    # create lines for numerical solution
    ll_n=[ax.plot(x,phi[frames[0],:,jExp],label=labels[jExp])[0] for jExp in range(nExp)]
    
    # set ylim
    ax.set_ylim(ylim)
    
    # add legend
    if showlegend:
        plt.legend()
    
    # add title
    tt=plt.title('')
    

    def animate(iframe):
        for jExp in range(nExp):
            ll_n[jExp].set_ydata(phi[iframe,:,jExp])
        tt.set_text('timestep %i/%i'%(iframe,nt))

    anim=animation.FuncAnimation(fig, animate, frames=frames,cache_frame_data=False,blit=True)
    display(anim)

## Effect of lateral boundary conditions


The effect of lateral boundary conditions is shown with the advection equation, applied to an initial state that can be tought of as a depression.

Boundary conditions are not periodic, but imposed on the endpoints of the domain. To mimic different situations, a coupling strength parameter $\alpha$ is introduced:
* $\alpha=0$ means no coupling at all: the values at the endpoints just keep their initial values.
* $\alpha=0.9$ mimicks coupling to a lower-resolution (global) model, which underestimates the depth of the depression
* $\alpha=1.0$ means imposing the exact solution (not possible for a real atmospheric for which the exact solution is unknown)


In [2]:

# leapfrog with 2nd order centered finite differences


# parameters
dt=60.
nt=100
nx=128
dx=1000.0
c=16

alpha=0.0   # coupling strength between limited-area model and exact solution

# define coordinates
x=np.arange(nx)*dx
t=np.arange(nt)*dt

def exact_solution(x):
    return(-np.exp(-100*(2*x/(nx*dx)-1.5)**2))

# allocate numerical solution, initialize with zeros
phi=np.zeros((nt+1,nx,2))     # phi[:,:,0] is exact solution; phi[:,:,1] is numerical solution
phi[0,:,0]=exact_solution(x)
phi[0,:,1]=exact_solution(x)

# first timestep: forward scheme
jt=0
phi[jt+1,:,0]=exact_solution(x-c*dt*(jt+1))
phi[jt+1,1:nx-1,1]=phi[jt,1:nx-1,1]-c*dt/(2*dx)*(phi[jt,2:nx,1]-phi[jt,0:nx-2,1])
# lbc's
phi[jt+1,0,1]=alpha*phi[jt+1,0,0]
phi[jt+1,nx-1,1]=alpha*phi[jt+1,nx-1,0]

# evolution in time: leapfrog
for jt in range(1,nt):
    phi[jt+1,:,0]=exact_solution(x-c*dt*(jt+1))
    phi[jt+1,1:nx-1,1]=phi[jt-1,1:nx-1,1]-c*dt/dx*(phi[jt,2:nx,1]-phi[jt,0:nx-2,1])
    # lbc's
    phi[jt+1,0,1]=alpha*phi[jt+1,0,0]
    phi[jt+1,nx-1,1]=alpha*phi[jt+1,nx-1,0]

create_animation(phi=phi,labels=['exact','numerical'],ylim=[-1.2,1.2])